# Glaberish

## A general framework for continuous CA that distinguishes genesis and persistence


### Overview

![](../assets/glaberish_s11_summary.png)

Glaberish is a continuous cellular automata (CA) framework that extends the functionality of Lena ([Chan 2019](https://www.complex-systems.com/abstracts/v28_i03_a01/)) to CA rules with non-overlapping conditions for cell growth and cell persistence. These comes from splitting the single growth function in Lenia into separate genesis and persistence functions, analogous to the birth and survival rules of CA from the family of CA known as 'Life-like', Conway's Game of Life being the seminal example. 

Whether implementing CA in the Lenia or Glaberish frameworks, it's useful to keep in mind that they both distill down to the same form as Euler's method. 

The dynamics in Lenia/Glaberish CA are described as

$$
A_{t+ \Delta t} = [A_t + \Delta t * U(A_t)]_0^1
$$

Where $A_t$ is the cell states in the CA universe (usually a 2D grid, sometimes with multiple channels), $\Delta t$ is the step size, and $U(A_t)$ is the update function. $A_t$ is typically squashed or truncated to values between 0 and 1.0, denoted by $[\cdot]_0^1$ above. In terms of the Euler method, $U(A_t)$ is the differential equation describing change in $A$, which is equivalent to the equation of interest $f(\cdot)$. The step size is often written as $h$ in Euler's method. 

$$
f(t + h) = f(t) + h * \frac{\partial f}{\partial t}
$$

In Lenia, the update function is called the growth function, which acts on the cell neighborhoods defined by convolution with a kernel $K_n$: $G_{growth}(K_n \circledast A_t)$. 

Glaberish is similar, but weights the genesis ($G$) and persistence ($P$) functions by 1.0 minus the cell states or the cell states (respectively). 

$$
\frac{\partial A}{\partial t} = U(A_t) = (1 - K_i \circledast A_t) G(K_n \circledast A_t) + (K_i \circledast A_t) P(K_n \circledast A_t)
$$

### Updates

Genesis and persistence can be arbitrary functions, but one form that works well in practice is that of a Gaussian function shifted to output values in a range from -1 to 1. A shifted genesis function with parameters $\mu_G$ (setting the peak) and $\sigma_G$ (determining the width of the Gaussian curve) looks like:

$$
G(x) = 2 {e} ^{- \frac{(x - \mu_G)^2}{2 \sigma_G^2} } - 1
$$

The persistence function takes the same form. When genesis and persistence have the same parameters (e.g. $\mu_G=\mu_P$ and $\sigma_G = \sigma_P$, the glaberish CA is equivalent to a Lenia CA with $G_{growth}=G=P$


### Neighborhoods

A convenient way to define neighborhood kernels is as a Gaussian function acting on the distance from the kernel center $r$. As for genesis/persistence/growth functions, $\mu_K$ and $\sigma_K$ determine the kernel specifics. 

$$
K_n(r)  = a {e} ^{- \frac{(r - \mu_K)^2}{2 \sigma_K^2} }
$$

$a$ is a normalization coefficient, equal to 1 over the sum of the un-normalized kernel. This ensures that neighborhood kernels can scale to more or fewer pixels without changing the magnitude of the resulting neighborhoods. 

$$ 
\frac{1}{a} = \sum_{i,j} e^{-\frac{(\sqrt{i^2+j^2} - b)^2}{c^2}} 
$$

This yields a ring neighborhood. Combining multiple weighted (radial) Gaussians each with their own $\mu$ and $\sigma$ makes for multiple-ring neighborhood kernels and interesting pattern morphologies.


![neighborhood kernels](../assets/neighborhood_kernels.png)


<!-- Image link to be used after making repo public -->
<!-- <img src="https://raw.github.com/riveSunder/DiscoGliders/assets/glaberish_4.gif"> -->

<!-- local image -->
![s11 gliders in Glaberish CA](../assets/glaberish_4.gif)

In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 128

import skimage
import skimage.io as sio
import skimage.transform

import yuca
from yuca.ca.neural import NCA
from yuca.ca.continuous import CCA
from yuca.cppn import CPPN

from yuca.zoo.librarian import Librarian
from yuca.kernels import get_kernel

torch.set_default_dtype(torch.float32)

import IPython

from importlib import reload
reload(yuca)
reload(yuca.ca)

In [ ]:
def plot_grid(grid, my_cmap=plt.get_cmap("magma"), title="Glaberish animation", vmin=0.0, vmax=1):

    global subplot_0
    
    fig, ax = plt.subplots(1,1, figsize=(4.5,4.5), facecolor="white")

    # TODO invert cmap
    
    my_cmap=plt.get_cmap("magma")
    grid_display = 1.0 - my_cmap(grid[0,0])[:,:,:3]
    
    subplot_0 = ax.imshow(grid_display, interpolation="nearest")
    
    fig.suptitle(title, fontsize=8)

    ax.set_yticklabels('')
    ax.set_xticklabels('')
    
    plt.tight_layout()

    return fig, ax

def update_fig(i):

    global subplot_0    
    global grid
    #global ax
    
    grid = ca(grid)
    
    my_cmap=plt.get_cmap("magma")
    grid_display = 1.0 - my_cmap(grid[0,0])[:,:,:3]
    
    subplot_0.set_array(grid_display)
        
    plt.tight_layout()

In [ ]:
lib = Librarian(verbose=False)

save_figs = True

num_frames = 420
grid_dim = 96

In [ ]:
# neighborhood kernels demo

ca = CCA()

p, m = lib.load("orbium_orbium000")

ca.restore_config(m["ca_config"])

plt.figure()
plt.subplot(121)
plt.imshow(ca.neighborhood_kernels.squeeze().detach(), cmap="magma")
plt.title("Gaussian")
#plt.supttitle("Neighborhood kernels")

p, m = lib.load("s11_config_evolved_ca_slow_glider000")

ca.restore_config(m["ca_config"])

plt.subplot(122)
plt.imshow(ca.neighborhood_kernels.squeeze().detach(), cmap="magma")
plt.title("3 Gaussians")
plt.tight_layout()
plt.show()

In [ ]:
pattern_name = "s11_config_evolved_ca_slow_glider000"


p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])

grid = ca.initialize_grid(dim=grid_dim)

grid[:,:,:p.shape[-2],:p.shape[-1]] = torch.tensor(p)

grid[:,:,32:32+p.shape[-2],-p.shape[-1]:] = torch.tensor(p.transpose(0,1,3,2))

fig, ax = plot_grid(grid)
plt.show()

if save_figs:
    save_path = os.path.join("..","assets", "glaberish_4.gif")
    matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).save(save_path)
else: IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).to_jshtml())